# Numerical optimization and large scale linear algebra

### Rallis Panagiotis p3351816

### Exercise 3 part 1

In [1]:
import numpy as np
from numpy import loadtxt
import scipy.sparse
from scipy.sparse import csr_matrix
import time

In [2]:
#load data and create sparse matrix A
data = np.loadtxt('stanweb.dat')
n = data[len(data)-1,0].astype(int)
row = data[:,0].astype(int)-1
column = data[:,1].astype(int)-1
propabilities = data[:,2]
A = csr_matrix((propabilities, (row, column)), shape=(n, n))

### a.i) The Power Method 

#### Υλοποιηση του αλγόριθμου power method χρησιμοποιώντας την παράσταση (1) σελίδα 7 paper Deeper Inside PageRank

In [3]:
def power_method(A , a = 0.85 , r = 10**-8, max_iter = 200):
    (n,d)=A.shape 
    zero_rows = A.sum(axis=1)==0
    index = np.argwhere(zero_rows)
    a_v = np.zeros(n)
    a_v[index[:,0]] = 1
    a_v = a_v.reshape(n,1)  
    
    vT = np.ones(d) / n            #initialiaze v
    
    x = np.ones(d) /  n 
    
#     iterations = np.log10(r)/np.log10(a)   # pagerank pdf page 8-9 Rate of Convergence
    
#     for i in range(0,int(round(iterations))):
    for i in range(max_iter):  
        
        x_new = a * x * A + (a * x * a_v.T + (1 - a)) * vT
        
        error = np.linalg.norm(x-x_new)
        x = x_new
        iters = i
        if error < r:
            x_new = x_new.reshape(n,1) 
            return x_new, iters
        
    x_new = x_new.reshape(n,1)       
    return x_new, iters

In [4]:
start1 = time.time()
x_power, power_iter = power_method(A , 0.85, 10**-8)
end1 = time.time()
runtime_power = end1-start1

Χρόνος εκτέλεσης

In [5]:
runtime_power

1.4373152256011963

Πλήθος επαναλήψεων

In [6]:
power_iter

71

In [7]:
x_power

array([[5.32097920e-07],
       [1.16899486e-04],
       [8.25192068e-07],
       ...,
       [5.35745366e-07],
       [1.80443667e-06],
       [1.47571468e-06]])

### a.ii) Solving the corresponding system with Gauss Seidel

In [8]:
from scipy import sparse
from scipy.sparse import linalg
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import inv
from scipy.linalg import tril
from scipy.sparse import csr_matrix, tril,triu
import time
from scipy.sparse.linalg import factorized,spsolve_triangular

#### Υλοποίηση τη μεθόδου Gauss Seidel χρησιμοποιόντας την class SuperLU scipy https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.linalg.SuperLU.html

In [9]:
#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.linalg.SuperLU.html
from scipy.sparse import csc_matrix, linalg as sla
def gauss_seidel(A, a = 0.85 , r = 10**-8, max_iter = 200):
    (n,m) = A.shape
    I = sparse.identity(n)
    
    A_temp = (I - a * A).T

    L = tril(A_temp, 0)
    U = triu(A_temp, 1)
    
    lu = sla.splu(L) 
    
    b =  np.ones(m) / n 
    x =  np.ones(m) /  n
    
#   iterations = np.log10(r)/np.log10(a)    
#   for i in range(0,int(round(iterations))): 

    for i in range(max_iter): 

        x_new = lu.solve(b - U*x)

        error = np.linalg.norm(x-x_new)
        
        x = x_new    
        iters = i
        if error < r:
            x_new = x_new.reshape(n,1) 
            return x_new / np.sum(x_new), iters
        
    return x_new/np.sum(x_new), iters


In [10]:
start1 = time.time()
x_gauss, gauss_iter =gauss_seidel(A , 0.85, 10**-8)
end1 = time.time()
runtime_gauss = end1 - start1

C:\Users\ralli\Anaconda3\envs\practical-data-science\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:295: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [11]:
runtime_gauss

2.6500866413116455

In [12]:
gauss_iter

49

In [13]:
runtime_power

1.4373152256011963

In [14]:
power_iter

71

#### Παρατηρούμε ότι ενώ ο χρόνος εκτέλεσης της Gauss Seidel είναι σχεδόν διπλάσιος από την Power Method, η μέθοδος Gauss Seidel χρειάζεται λιγότερες επαναλήψεις από την Power Method.

##### Generally speaking, we expect the solution of these subproblems to take fewer iterations individually than is required for the huge matrix 2, leading to less total arithmetic. The other payoff, perhaps the payoff, is in reduced I/O
#### [7] Arvind Arasu, Jasmine Novak, Andrew Tomkins, and John Tomlin. PageRank computation and the structure of the Web: experiments and algorithms. In The Eleventh International WWW Conference, New York, May 2002. ACM Press. http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=8A501558ABCD222947A10486EF0D08A9?doi=10.1.1.387.7294&rep=rep1&type=pdf

In [15]:
import pandas as pd
df = pd.DataFrame({'Power Method Prob':x_power[:,0],'Gauss Seidel Prob':x_gauss[:,0]})
df=df.sort_values(by=['Power Method Prob'], ascending = False)
df

,Power Method Prob,Gauss Seidel Prob
89072,1.127596e-02,1.130284e-02
226410,9.265573e-03,9.287656e-03
241453,8.277506e-03,8.297236e-03
262859,3.015929e-03,3.023117e-03
134831,2.994130e-03,3.001267e-03
234703,2.566140e-03,2.572256e-03
136820,2.447868e-03,2.453703e-03
68888,2.425000e-03,2.430780e-03
105606,2.391727e-03,2.397428e-03
69357,2.358382e-03,2.364004e-03


#### Παρατηρούμε ότι τα αποτελέσματα είναι ίδια για τις 2 μεθόδους με μικρές αποκλίσεις στις πιθανότητες. Ωστόσο ο χρόνος εκτέλεσης της Gauss Siedel είναι σχεδόν διπλάσιος από την Power Method. 

### b) Repeat above methods with a=0.99

In [16]:
start1 = time.time()
x_power_b, power_iter_b  = power_method(A , 0.99, 10**-8, 2000 )
end1 = time.time()
runtime_power_b = end1-start1

In [17]:
x_power_b

array([[3.54731947e-08],
       [1.02639271e-04],
       [1.28503451e-07],
       ...,
       [3.57564081e-08],
       [4.03580480e-07],
       [1.85020355e-07]])

In [18]:
runtime_power_b

22.07921028137207

In [19]:
power_iter_b

1140

In [20]:
start1 = time.time()
x_gauss_b, gauss_iter_b=gauss_seidel(A , 0.99, 10**-8, 2000)
end1 = time.time()
runtime_gauss_b = end1 - start1

C:\Users\ralli\Anaconda3\envs\practical-data-science\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:295: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [21]:
runtime_gauss_b

37.51128840446472

In [22]:
gauss_iter_b

814

In [23]:
import pandas as pd
df_b = pd.DataFrame({'Power Method Prob':x_power_b[:,0],'Gauss Seidel Prob':x_gauss_b[:,0]})
df_b=df_b.sort_values(by=['Power Method Prob'], ascending = False)
df_b

,Power Method Prob,Gauss Seidel Prob
89072,9.020728e-03,9.186779e-03
281771,8.947687e-03,9.112392e-03
174664,7.549560e-03,7.688528e-03
226410,4.432867e-03,4.514466e-03
179644,3.999107e-03,4.072721e-03
271408,3.802341e-03,3.872332e-03
262859,3.422333e-03,3.485331e-03
136820,2.769847e-03,2.820834e-03
68888,2.739781e-03,2.790214e-03
77987,2.627871e-03,2.676245e-03


In [24]:
df_b.index[:50] == df.index[:50] 

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

#### Παρατηρούμε ότι το ranking των πρώτων 50 κόμβων άλλαξε για 48 από τους 50 κόμβους. Επίσης όπως αναμενόταν ο χρόνος εκτέλεσης των δυο μεθόδων σχεδόν δεκαπλάσιαστηκε.
#### Η συγκεκριμένη συμπεριφορά ήταν αναμενόμενη και επιβεβαιώνεται στην παράγραφο 6.1 Changing α του paper Deeper Inside PageRank.
##### The PageRank vector derived from α = .99 can be vastly different from that obtained using α = .85. Perhaps it gives a “truer” PageRanking.

## c)   When we use the power method do all the components of π converge at the same speed to their limits? If not which of the converge faster: those that correspond to important nodes or to non important ? Do you observe the same behavior when you and π through the solution of the linear system?

#### Σύμφωνα με το paper [69] Sepandar D. Kamvar, Taher H. Haveliwala, and Gene H. Golub. Adaptive methods for the computation of PageRank. Technical Report 2003-26, Stanford University, 2003. http://infolab.stanford.edu/~taherh/papers/adaptive.pdf

#####  The convergence rates of the PageRank values of individual pages during application of the Power Method is nonuniform. That is, many pages converge quickly, with a few pages taking much longer to converge. Furthermore, the pages that converge slowly are generally those pages with high PageRank.

##### Το PageRank Xi μιας σελίδας i συγκλίνει όταν: |Χi(k+1) - Xi(k)| / |Xi(k)| < 10^-3

#### Προστέθηκε στις δυο μεθόδους o παραπάνω έλεγχος. Συγκεκριμένα επιστρέφω την επανάληψη τη στιγμή που η παραπάνω συνθήκη ικανοποιείται για τις 10 μεγαλύτερες τιμές και για τις 10 μικρότερες.

In [25]:
def power_method_2(A , a = 0.85 , r = 10**-8, max_iter = 200):
    (n,d)=A.shape 
    zero_rows = A.sum(axis=1)==0
    index = np.argwhere(zero_rows)
    a_v = np.zeros(n)
    a_v[index[:,0]] = 1
    a_v = a_v.reshape(n,1)  
    
    vT = np.ones(d) / n            #initialiaze v
    
    x = np.ones(d) /  n 
    
#     iterations = np.log10(r)/np.log10(a)   # pagerank pdf page 8-9 Rate of Convergence
    
#     for i in range(0,int(round(iterations))):
    limitd = 10**-3
    limita = 10**-3
    for i in range(max_iter):  
        
        x_new = a * x * A + (a * x * a_v.T + (1 - a)) * vT
        
        x_new_sort_desc = -np.sort(-x_new.ravel())
        x_sort_desc     = -np.sort(-x.ravel())
        x_new_sort_asc  =  np.sort( x_new.ravel())
        x_sort_asc      =  np.sort( x.ravel())        
        
        converge_desc = np.mean(np.abs(x_new_sort_desc[:10] - x_sort_desc[:10]) / np.abs(x_sort_desc[:10]))
        
        converge_asc = np.mean(np.abs(x_new_sort_asc[:10] - x_sort_asc[:10]) / np.abs(x_sort_asc[:10]))
        
        
        if converge_desc < limitd: 
            print ('Iterations for important:'+ str(i+1) ) # +1 γιατί η πρώτη επανάληψη είναι η μηδέν
            limitd = -1000
        if converge_asc < limita: 
            print ('Iterations for non important:'+ str(i+1) )
            limita = -1000    
        
        
        error = np.linalg.norm(x-x_new)
        x = x_new
        iters = i
        if error < r:
            x_new = x_new.reshape(n,1) 
            return x_new, iters
        
    x_new = x_new.reshape(n,1)       
    return x_new, iters

In [26]:
x_power_c, power_iter_c  = power_method_2(A , 0.85, 10**-8, 2000 )

Iterations for non important:2
Iterations for important:16


In [27]:
#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.linalg.SuperLU.html
from scipy.sparse import csc_matrix, linalg as sla
def gauss_seidel_2(A, a = 0.85 , r = 10**-8, max_iter = 200):
    (n,m) = A.shape
    I = sparse.identity(n)
    
    A_temp = (I - a * A).T

    L = tril(A_temp, 0)
    U = triu(A_temp, 1)
    
    lu = sla.splu(L) 
    
    b =  np.ones(m) / n 
    x =  np.ones(m) /  n
    
#   iterations = np.log10(r)/np.log10(a)    
#   for i in range(0,int(round(iterations))): 
    limitd = 10**-3
    limita = 10**-3
    for i in range(max_iter): 

        x_new = lu.solve(b - U*x)
        
        x_new_sort_desc = -np.sort(-x_new.ravel())
        x_sort_desc     = -np.sort(-x.ravel())
        x_new_sort_asc  =  np.sort( x_new.ravel())
        x_sort_asc      =  np.sort( x.ravel())        
        
        converge_desc = np.mean(np.abs(x_new_sort_desc[:10] - x_sort_desc[:10]) / np.abs(x_sort_desc[:10]))
        
        converge_asc = np.mean(np.abs(x_new_sort_asc[:10] - x_sort_asc[:10]) / np.abs(x_sort_asc[:10]))
        
        
        if converge_desc < limitd: 
            print ('Iterations for important:'+ str(i+1) ) # +1 γιατί η πρώτη επανάληψη είναι η μηδέν
            limitd = -1000
        if converge_asc < limita: 
            print ('Iterations for non important:'+ str(i+1) )
            limita = -1000    
        
        

        error = np.linalg.norm(x-x_new)
        
        x = x_new    
        iters = i
        if error < r:
            x_new = x_new.reshape(n,1) 
            return x_new / np.sum(x_new), iters
        
    return x_new/np.sum(x_new), iters


In [28]:
x_gauss_c, gauss_iter_c  = gauss_seidel_2(A , 0.85, 10**-8, 2000 )

C:\Users\ralli\Anaconda3\envs\practical-data-science\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:295: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Iterations for non important:1
Iterations for important:18


#### Επιβεβαιώνουμε λοιπόν ότι και στις δύο μεθόδους, οι σελίδες με μεγάλο PageRank χρειάζονται περισσότερες επαναλήψεις για να συγκλίνουν από τις λιγότερο σημαντικές σελίδες